### OpenAPI(Swagger) -> React TypeScript types.ts 생성
- .env의 API_URL에서 주소를 읽고, 모든 paths의 요청/응답 타입 + components.schemas를 types.ts로 생성

In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
OpenAPI/Swagger -> React TypeScript types.ts 생성기 (NO 'any', multipart/form-data 방어 포함)

기능
- 환경변수로 스펙 URL 지정:
  * API_SPEC_URLS (쉼표로 여러 개)
  * 또는 API_SPEC_URL | API_URL | SWAGGER_URL (단일)
- swagger-ui.html 같은 UI 주소여도 /v3/api-docs, /v3/api-docs.yaml, /v2/api-docs,
  /openapi.{json,yaml} 후보로 자동 보정
- OpenAPI 3 / Swagger v2 모두 파싱, 여러 스펙 병합
- 엔드포인트별 Params / RequestBody / Response 타입 생성
- 'any' 미사용: 모르는 타입은 'unknown', 응답 없음은 'void'
- multipart/form-data / x-www-form-urlencoded 방어:
  - 스키마가 있으면 그대로 사용
  - 스키마가 없으면 안전한 폴백(문자/숫자/불리언/파일/파일배열/null 허용) 사용
사용
  pip install requests pyyaml
  export API_SPEC_URLS="http://localhost:8080/swagger-ui/index.html, https://api.example.com/v3/api-docs"
  # 선택: 인증
  # export API_AUTH_BEARER="eyJhbGciOi..."
  # export API_AUTH_BASIC="user:pass"
  # export API_AUTH_HEADER="X-API-Key: abc123\nX-Tenant: demo"
  # 출력 파일
  # export OUT_TYPES="src/types.ts"
  python openapi_to_types_no_any_multipart.py
"""

import json
import os
import re
import textwrap
import base64
import pathlib
from typing import Any, Dict, List, Optional
from collections import OrderedDict
from dotenv import load_dotenv
load_dotenv()

try:
    import requests
except Exception:
    raise RuntimeError("Please install requests: pip install requests")
try:
    import yaml
except Exception:
    raise RuntimeError("Please install pyyaml: pip install pyyaml")

# ---------------------- Auth headers from env ----------------------
def _auth_headers_from_env() -> Dict[str, str]:
    """
    선택적 인증 헤더 구성:
      - API_AUTH_BEARER="xxxxx"  -> Authorization: Bearer xxxxx
      - API_AUTH_BASIC="user:pass" -> Authorization: Basic base64(user:pass)
      - API_AUTH_HEADER="Header: value" (여러 줄 지원)
    """
    headers: Dict[str, str] = {}
    bearer = os.getenv("API_AUTH_BEARER", "").strip()
    if bearer:
        headers["Authorization"] = f"Bearer {bearer}"
    basic = os.getenv("API_AUTH_BASIC", "").strip()
    if basic and ":" in basic:
        token = base64.b64encode(basic.encode("utf-8")).decode("ascii")
        headers["Authorization"] = f"Basic {token}"
    extra = os.getenv("API_AUTH_HEADER", "").strip()
    if extra:
        for line in extra.splitlines():
            if ":" in line:
                k, v = line.split(":", 1)
                headers[k.strip()] = v.strip()
    return headers

# ---------------------- URL guessing (UI -> spec) ----------------------
UI_SUFFIXES = (
    "/swagger-ui.html",
    "/swagger-ui/index.html",
    "/swagger-ui",
    "/swagger",
    "/docs",
    "/redoc",
)

def _guess_spec_url(ui_url: str) -> List[str]:
    """swagger-ui 같은 UI 주소를 실제 스펙 주소 후보로 변환"""
    base = ui_url.rstrip("/")
    candidates: List[str] = []
    for suf in UI_SUFFIXES:
        if base.endswith(suf):
            root = base[: -len(suf)]
            candidates += [
                root + "/v3/api-docs",
                root + "/v3/api-docs.yaml",
                root + "/v2/api-docs",
                root + "/openapi.json",
                root + "/openapi.yaml",
            ]
            break
    if not candidates:
        candidates = [
            base + "/v3/api-docs",
            base + "/v3/api-docs.yaml",
            base + "/v2/api-docs",
            base + "/openapi.json",
            base + "/openapi.yaml",
        ]
    seen = set(); out: List[str] = []
    for c in candidates:
        if c not in seen:
            out.append(c); seen.add(c)
    return out

# ---------------------- Fetch JSON or YAML ----------------------
def _fetch_json_or_yaml(url: str, headers: Dict[str, str]) -> Optional[Dict[str, Any]]:
    r = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
    r.raise_for_status()
    text = r.text
    # Try JSON
    try:
        obj = json.loads(text)
        if isinstance(obj, dict):
            return obj
    except Exception:
        pass
    # Try YAML
    try:
        obj = yaml.safe_load(text)
        if isinstance(obj, dict):
            return obj
    except Exception:
        pass
    return None

# ---------------------- Load multiple specs from env ----------------------
def load_all_specs_from_env() -> List[Dict[str, Any]]:
    """
    환경변수에서 스펙 URL들을 읽어 모두 로드:
    - API_SPEC_URLS(쉼표 구분) 또는 API_SPEC_URL | API_URL | SWAGGER_URL 단일
    - UI 주소면 후보 스펙 주소로 자동 보정
    """
    raw = (
        os.getenv("API_SPEC_URLS") or
        os.getenv("API_SPEC_URL") or
        os.getenv("API_URL") or
        os.getenv("SWAGGER_URL") or
        ""
    ).strip()
    if not raw:
        raise RuntimeError("Set API_SPEC_URLS (comma-separated) or API_SPEC_URL | API_URL | SWAGGER_URL in env.")
    urls = [u.strip() for u in raw.split(",") if u.strip()]
    headers = _auth_headers_from_env()
    specs: List[Dict[str, Any]] = []
    for url in urls:
        obj = _fetch_json_or_yaml(url, headers)
        if not (isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger"))):
            for cand in _guess_spec_url(url):
                obj = _fetch_json_or_yaml(cand, headers)
                if isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger")):
                    break
        if not (isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger"))):
            raise RuntimeError(f"Failed to load spec from: {url}")
        specs.append(obj)
    return specs

# ---------------------- Merge specs ----------------------
def _deep_merge(dst: Dict[str, Any], src: Dict[str, Any]) -> None:
    for k, v in src.items():
        if isinstance(v, dict) and isinstance(dst.get(k), dict):
            _deep_merge(dst[k], v)
        else:
            dst[k] = v

def merge_openapi_specs(specs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    여러 스펙 병합 (뒤에 오는 것이 우선)
    - paths, components(v3), definitions(v2), parameters/responses/security...
    """
    if not specs:
        raise RuntimeError("No specs to merge.")
    out = json.loads(json.dumps(specs[0]))  # deep copy
    for s in specs[1:]:
        _deep_merge(out.setdefault("paths", {}), s.get("paths", {}))
        if "components" in s:
            _deep_merge(out.setdefault("components", {}), s["components"])
        if "definitions" in s:  # Swagger v2
            _deep_merge(out.setdefault("definitions", {}), s["definitions"])
        for k in ("parameters", "responses", "securityDefinitions", "securitySchemes", "tags"):
            if k in s:
                if isinstance(s[k], dict):
                    _deep_merge(out.setdefault(k, {}), s[k])
                else:
                    out[k] = s[k]
    return out

# ---------------------- $ref resolver (safe) ----------------------
class RefResolver:
    """
    안전한 resolve:
    - 내부 $ref(#/...)만 즉시 대상 dict를 반환
    - 전체 딥-전개는 메모이제이션으로 순환 방지
    - 외부 $ref는 그대로 둠
    """
    def __init__(self, spec: Dict[str, Any]):
        self.spec = spec
        self._cache: Dict[int, Any] = {}

    def _get_by_pointer(self, tokens: List[str]) -> Any:
        cur: Any = self.spec
        for tk in tokens:
            if not isinstance(cur, dict) or tk not in cur:
                return None
            cur = cur[tk]
        return cur

    def resolve(self, node: Any) -> Any:
        if not isinstance(node, dict):
            return node
        if id(node) in self._cache:
            return self._cache[id(node)]
        # placeholder to break cycles
        out: Dict[str, Any] = {}
        self._cache[id(node)] = out
        if "$ref" in node:
            ref = node["$ref"]
            if isinstance(ref, str) and ref.startswith("#/"):
                target = self._get_by_pointer(ref[2:].split("/"))
                return target if isinstance(target, dict) else node
            return node  # external ref untouched
        for k, v in node.items():
            if isinstance(v, dict):
                out[k] = self.resolve(v)
            elif isinstance(v, list):
                out[k] = [self.resolve(x) if isinstance(x, dict) else x for x in v]
            else:
                out[k] = v
        return out

# ---------------------- Type mapping (NO any) ----------------------
PRIMITIVES = {
    ("string",  None): "string",
    ("integer", None): "number",
    ("number",  None): "number",
    ("boolean", None): "boolean",
    ("null",    None): "null",
}
STRING_FORMATS = {
    "date-time": "string",
    "date": "string",
    "uuid": "string",
    "email": "string",
    "uri": "string",
    "binary": "Blob | File",  # multipart 파일
    "byte": "string",
    "password": "string",
}
FALLBACK_SCALAR = "unknown"
FALLBACK_INDEX  = "unknown"

def to_pascal_case(s: str) -> str:
    parts = re.split(r"[^A-Za-z0-9]+", s)
    return "".join(p[:1].upper() + p[1:] for p in parts if p)

def sanitize_ts_identifier(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9_]", "_", s)
    if re.match(r"^\d", s):
        s = "_" + s
    return s

# ---------------------- Schema -> TS (depth guard) ----------------------
MAX_DEPTH = 40  # 필요 시 조정

def ts_type_for_schema(
    name_hint,
    schema,
    components,
    resolver,
    decls,
    depth: int = 0,
) -> str:
    # 깊이 가드
    if depth > MAX_DEPTH:
        return "unknown /* depth capped */"

    if not isinstance(schema, dict):
        return FALLBACK_SCALAR

    # $ref는 전개하지 않고 참조명 그대로 반환 (순환 방지)
    if "$ref" in schema:
        ref = schema["$ref"]
        ref_name = ref.split("/")[-1] if isinstance(ref, str) else "UnknownRef"
        return sanitize_ts_identifier(ref_name)

    typ = schema.get("type")
    fmt = schema.get("format")
    enum = schema.get("enum")
    nullable = bool(schema.get("nullable", False))

    # enum
    if enum:
        lit = " | ".join(json.dumps(e, ensure_ascii=False) for e in enum)
        return f"({lit})" + (" | null" if nullable else "")

    # unions
    if "oneOf" in schema:
        union = " | ".join(
            ts_type_for_schema(name_hint, s, components, resolver, decls, depth + 1)
            for s in schema["oneOf"]
        )
        return f"({union})" + (" | null" if nullable else "")

    if "anyOf" in schema:
        union = " | ".join(
            ts_type_for_schema(name_hint, s, components, resolver, decls, depth + 1)
            for s in schema["anyOf"]
        )
        return f"({union})" + (" | null" if nullable else "")

    if "allOf" in schema:
        parts = schema["allOf"]
        items = [
            ts_type_for_schema(name_hint, p, components, resolver, decls, depth + 1)
            for p in parts
        ]
        inter = " & ".join(items) if items else "unknown"
        return inter + (" | null" if nullable else "")

    # scalars
    if typ in ("string", "integer", "number", "boolean", "null"):
        if typ == "string" and fmt and fmt in STRING_FORMATS:
            ts = STRING_FORMATS[fmt]
        else:
            ts = PRIMITIVES.get((typ, None), FALLBACK_SCALAR)
        return ts + (" | null" if nullable else "")

    # arrays
    if typ == "array":
        inner = ts_type_for_schema(
            name_hint + "Item", schema.get("items", {}), components, resolver, decls, depth + 1
        )
        return f"{inner}[]" + (" | null" if nullable else "")

    # objects (resolve 사용 금지)
    if typ == "object" or ("properties" in schema) or ("additionalProperties" in schema):
        props = schema.get("properties", {}) or {}
        req = set(schema.get("required", []))
        lines: List[str] = []

        for prop_name, prop_schema in props.items():
            ts_prop = ts_type_for_schema(
                to_pascal_case(f"{name_hint}_{prop_name}"),
                prop_schema,                # <- resolver.resolve 제거
                components, resolver, decls, depth + 1
            )
            optional = "?" if prop_name not in req else ""
            safe_key = sanitize_ts_identifier(prop_name)
            lines.append(f"  {safe_key}{optional}: {ts_prop};")

        addl = schema.get("additionalProperties")
        index_sig = ""
        if addl is True:
            index_sig = f"  [key: string]: {FALLBACK_INDEX};\n"
        elif isinstance(addl, dict):
            t_addl = ts_type_for_schema(
                name_hint + "Additional", addl, components, resolver, decls, depth + 1  # <- resolve 제거
            )
            index_sig = f"  [key: string]: {t_addl};\n"

        iface_name = sanitize_ts_identifier(to_pascal_case(name_hint) or "Unnamed")
        decl = "export interface " + iface_name + " {\n" + ("\n".join(lines) + ("\n" if lines else "")) + index_sig + "}"
        if decl not in decls:
            decls.append(decl)
        return iface_name + (" | null" if nullable else "")

    if "title" in schema:
        return sanitize_ts_identifier(to_pascal_case(schema["title"])) + (" | null" if nullable else "")

    # unknown
    return FALLBACK_SCALAR + (" | null" if nullable else "")

# ---------------------- Helpers for media types ----------------------
JSON_CTYPES = (
    "application/json",
    "application/ld+json",
    "text/json",
    "application/problem+json",
)
FORM_CTYPES = (
    "multipart/form-data",
    "application/x-www-form-urlencoded",
)

def _first_schema_preferring(ctypes: List[str], content: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    for ct in ctypes:
        if ct in content and isinstance(content[ct], dict):
            media = content[ct]
            sch = media.get("schema")
            if sch:
                # 원본 schema 그대로 반환(전개하지 않음)
                return sch
    return None

def _fallback_multipart_schema() -> Dict[str, Any]:
    """multipart/form-data, x-www-form-urlencoded 에 스키마가 전혀 없을 때의 안전한 폴백"""
    return {
        "type": "object",
        "additionalProperties": {
            "oneOf": [
                {"type": "string"},
                {"type": "number"},
                {"type": "boolean"},
                {"type": "string", "format": "binary"},                         # Blob | File
                {"type": "array", "items": {"type": "string", "format": "binary"}},  # (Blob|File)[]
                {"type": "null"}
            ]
        }
    }

# ---------------------- request/response/params 추출 ----------------------
def first_response_schema(op: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """응답 스키마 선택: 2xx 우선 → JSON 우선 → 첫 미디어 스키마"""
    responses = op.get("responses", {}) or {}
    two_xx = [k for k in responses.keys() if str(k).startswith("2")]
    ordered_keys = sorted(two_xx) + [k for k in responses.keys() if k not in two_xx]
    for code in ordered_keys:
        resp_obj = responses.get(code, {}) or {}
        content = resp_obj.get("content", {}) or {}
        sch = _first_schema_preferring(list(JSON_CTYPES), content)
        if sch:
            return sch
        for media in content.values():
            if isinstance(media, dict) and "schema" in media and media["schema"]:
                return media["schema"]
    return None

def params_schema(op: Dict[str, Any], resolver: RefResolver) -> Dict[str, Any]:
    """parameters(path/query/header/cookie)를 하나의 객체로 묶는 유사 스키마. 스키마 없으면 string"""
    params = op.get("parameters", []) or []
    props: Dict[str, Any] = {}; required: List[str] = []
    for p in params:
        # 파라미터는 $ref로 자주 쓰이므로 안전 resolve
        p = resolver.resolve(p)
        name = p.get("name")
        sch = p.get("schema") or {"type": "string"}  # default string
        if not name:
            continue
        props[name] = sch
        if p.get("required"):
            required.append(name)
    return {"type": "object", "properties": props, "required": required}

def request_body_schema(op: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    requestBody 스키마 선택:
      1) JSON 우선
      2) multipart/x-www-form-urlencoded (스키마 없으면 안전폴백)
      3) 기타 첫 스키마
    """
    body = op.get("requestBody")
    if not body:
        return None
    content = (body.get("content") or {})
    sch = _first_schema_preferring(list(JSON_CTYPES), content)
    if sch:
        return sch
    # form
    for ct in FORM_CTYPES:
        if ct in content and isinstance(content[ct], dict):
            media = content[ct]
            if isinstance(media, dict) and "schema" in media and media["schema"]:
                return media["schema"]
            return _fallback_multipart_schema()
    # 기타
    for media in content.values():
        if isinstance(media, dict) and "schema" in media and media["schema"]:
            return media["schema"]
    return None

# ---------------------- Emit TS ----------------------
HTTP_METHODS = {"get", "post", "put", "patch", "delete", "head", "options", "trace"}

def emit_components_schemas(components: Dict[str, Any], resolver: RefResolver, decls: List[str]) -> None:
    """components.schemas를 순회하여 타입/인터페이스를 생성"""
    schemas = components.get("schemas", {}) if components else {}
    for raw_name, schema in schemas.items():
        name = sanitize_ts_identifier(to_pascal_case(raw_name))
        # 내부 $ref는 ts_type_for_schema가 이름으로 처리
        ts_expr = ts_type_for_schema(name, schema, components, resolver, decls)
        if ts_expr != name:
            alias = f"export type {name} = {ts_expr};"
            if alias not in decls:
                decls.append(alias)

def emit_paths(spec: Dict[str, Any], resolver: RefResolver, decls: List[str]) -> None:
    comp = spec.get("components", {})
    paths = spec.get("paths", {}) or {}
    for raw_path, methods in paths.items():
        if not isinstance(methods, dict):
            continue
        for m, op in methods.items():
            if m.lower() not in HTTP_METHODS or not isinstance(op, dict):
                continue
            op_id = op.get("operationId") or f"{m}_{raw_path}"
            base_name = sanitize_ts_identifier(to_pascal_case(op_id))

            # Params
            pschema = params_schema(op, resolver)
            ptype = ts_type_for_schema(base_name + "Params", pschema, comp, resolver, decls)
            if ptype != (base_name + "Params"):
                decls.append(f"export type {base_name}Params = {ptype};")

            # RequestBody
            rschema = request_body_schema(op)
            if rschema:
                rtype = ts_type_for_schema(base_name + "RequestBody", rschema, comp, resolver, decls)
                if rtype != (base_name + "RequestBody"):
                    decls.append(f"export type {base_name}RequestBody = {rtype};")
            else:
                decls.append(f"export type {base_name}RequestBody = undefined;")

            # Response
            res_schema = first_response_schema(op)
            if res_schema:
                rtype2 = ts_type_for_schema(base_name + "Response", res_schema, comp, resolver, decls)
                if rtype2 != (base_name + "Response"):
                    decls.append(f"export type {base_name}Response = {rtype2};")
            else:
                decls.append(f"export type {base_name}Response = void;")

            decls.append(f"// [{m.upper()}] {raw_path} -> {base_name}Params, {base_name}RequestBody, {base_name}Response")

# ---------------------- Build & Validate ----------------------
STRICT_NO_ANY = True

def assert_no_any(ts_code: str):
    """
    타입으로 사용된 any만 탐지한다.
    허용: 식별자/프로퍼티 이름 'any', 문자열 "any", 주석 속 any 등.
    탐지: ': any', '| any', '= any', '<any>' 같은 타입 문맥.
    """
    # 타입 문맥 패턴 (콜론/파이프/제네릭/할당 등 뒤에 오는 any)
    pattern = re.compile(r'(?x)'
                         r'(?:[:<|=]\s*any(?=[\s,;>\]\)&]))'   # : any, | any, = any, <any>
                         )
    if pattern.search(ts_code):
        raise RuntimeError("'any' detected **as a type** in output. Adjust mappings or specs.")

def build_types_ts(spec: Dict[str, Any]) -> str:
    resolver = RefResolver(spec)
    decls: List[str] = []
    header = textwrap.dedent("""
    /* AUTO-GENERATED by OpenAPI -> TS generator (Python)
     * - Merged multiple specs
     * - No 'any' (uses 'unknown' fallback)
     * - Supports JSON, multipart/form-data, x-www-form-urlencoded
     */
    """).strip()
    comp = spec.get("components", {})
    emit_components_schemas(comp, resolver, decls)
    emit_paths(spec, resolver, decls)
    body = "\n\n".join(OrderedDict.fromkeys(decls))
    code = header + "\n\n" + body + "\n"
    if STRICT_NO_ANY:
        assert_no_any(code)
    return code

def save_types_ts(ts_code: str, out_path: str = "types.ts"):
    pathlib.Path(out_path).write_text(ts_code, encoding="utf-8")
    print(f"[ok] wrote {out_path} ({len(ts_code)} bytes)")

# ---------------------- Main ----------------------
if __name__ == "__main__" or True:
    specs = load_all_specs_from_env()
    spec = merge_openapi_specs(specs)
    ts = build_types_ts(spec)
    out = os.getenv("OUT_TYPES", "types.ts")
    save_types_ts(ts, out)
    print("\n---- preview ----")
    print("\n".join(ts.splitlines()[:80]))

[ok] wrote types.ts (101998 bytes)

---- preview ----
/* AUTO-GENERATED by OpenAPI -> TS generator (Python)
 * - Merged multiple specs
 * - No 'any' (uses 'unknown' fallback)
 * - Supports JSON, multipart/form-data, x-www-form-urlencoded
 */

export interface ProfileDto {
  id?: number;
  accountId: number;
  locationId?: number;
  desiredJobCategoryId?: number;
  experienceLevel?: number;
  educationLevel?: number;
  skills?: string;
  desiredLocationId?: number;
  desiredSalaryCode?: number;
  profileImageKey?: string;
  updatedAt?: string;
}

export interface ResultDataProfileDto {
  count?: number;
  msg?: string;
  data?: ProfileDto;
}

export interface ResultDataObjectData {
}

export interface ResultDataObject {
  count?: number;
  msg?: string;
  data?: ResultDataObjectData;
}

export interface ResetPasswordRequestDTO {
  verificationToken?: string;
  code?: string;
  newPassword?: string;
}

export interface ResetPasswordResponseDTO {
  reset?: boolean;
}

export interface Res

## version 2

In [14]:
"""
OpenAPI/Swagger -> React TypeScript types2.ts 생성기
- NO 'any' (unknown fallback + 검증)
- multipart/form-data / x-www-form-urlencoded 방어
- $ref 전개 금지(순환안전) + 깊이 제한
- 동일 구조 스키마 자동 재사용(중복 인터페이스 제거)

환경변수:
  API_SPEC_URLS="https://host/swagger-ui/index.html, https://host/v3/api-docs"
  # 또는 API_SPEC_URL / API_URL / SWAGGER_URL 단일
  API_AUTH_BEARER / API_AUTH_BASIC / API_AUTH_HEADER  # 선택
  OUT_TYPES="src/types2.ts"                           # 출력 경로(선택)
"""

import json, os, re, textwrap, base64, pathlib, hashlib
from typing import Any, Dict, List, Optional, Tuple
from collections import OrderedDict
from dotenv import load_dotenv
load_dotenv()

try:
    import requests
except Exception:
    raise RuntimeError("Please install requests: pip install requests")
try:
    import yaml
except Exception:
    raise RuntimeError("Please install pyyaml: pip install pyyaml")

# ---------------------- Auth headers from env ----------------------
def _auth_headers_from_env() -> Dict[str, str]:
    headers: Dict[str, str] = {}
    b = os.getenv("API_AUTH_BEARER", "").strip()
    if b: headers["Authorization"] = f"Bearer {b}"
    basic = os.getenv("API_AUTH_BASIC", "").strip()
    if basic and ":" in basic:
        token = base64.b64encode(basic.encode("utf-8")).decode("ascii")
        headers["Authorization"] = f"Basic {token}"
    extra = os.getenv("API_AUTH_HEADER", "").strip()
    if extra:
        for line in extra.splitlines():
            if ":" in line:
                k, v = line.split(":", 1)
                headers[k.strip()] = v.strip()
    return headers

# ---------------------- UI -> spec URL 후보 ----------------------
UI_SUFFIXES = ("/swagger-ui.html","/swagger-ui/index.html","/swagger-ui","/swagger","/docs","/redoc")
def _guess_spec_url(ui_url: str) -> List[str]:
    base = ui_url.rstrip("/")
    cands: List[str] = []
    for suf in UI_SUFFIXES:
        if base.endswith(suf):
            root = base[:-len(suf)]
            cands += [root+"/v3/api-docs", root+"/v3/api-docs.yaml", root+"/v2/api-docs", root+"/openapi.json", root+"/openapi.yaml"]
            break
    if not cands:
        cands = [base+"/v3/api-docs", base+"/v3/api-docs.yaml", base+"/v2/api-docs", base+"/openapi.json", base+"/openapi.yaml"]
    seen=set(); out=[]
    for c in cands:
        if c not in seen: out.append(c); seen.add(c)
    return out

# ---------------------- Fetch JSON or YAML ----------------------
def _fetch_json_or_yaml(url: str, headers: Dict[str, str]) -> Optional[Dict[str, Any]]:
    r = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
    r.raise_for_status()
    text = r.text
    try:
        obj = json.loads(text)
        if isinstance(obj, dict): return obj
    except Exception:
        pass
    try:
        obj = yaml.safe_load(text)
        if isinstance(obj, dict): return obj
    except Exception:
        pass
    return None

# ---------------------- Load all specs ----------------------
def load_all_specs_from_env() -> List[Dict[str, Any]]:
    raw = (os.getenv("API_SPEC_URLS") or os.getenv("API_SPEC_URL") or os.getenv("API_URL") or os.getenv("SWAGGER_URL") or "").strip()
    if not raw:
        raise RuntimeError("Set API_SPEC_URLS (comma-separated) or API_SPEC_URL | API_URL | SWAGGER_URL.")
    urls = [u.strip() for u in raw.split(",") if u.strip()]
    headers = _auth_headers_from_env()
    specs: List[Dict[str, Any]] = []
    for url in urls:
        obj = _fetch_json_or_yaml(url, headers)
        if not (isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger"))):
            for cand in _guess_spec_url(url):
                obj = _fetch_json_or_yaml(cand, headers)
                if isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger")):
                    break
        if not (isinstance(obj, dict) and (obj.get("openapi") or obj.get("swagger"))):
            raise RuntimeError(f"Failed to load spec from: {url}")
        specs.append(obj)
    return specs

# ---------------------- Merge specs ----------------------
def _deep_merge(dst: Dict[str, Any], src: Dict[str, Any]) -> None:
    for k, v in src.items():
        if isinstance(v, dict) and isinstance(dst.get(k), dict):
            _deep_merge(dst[k], v)
        else:
            dst[k] = v

def merge_openapi_specs(specs: List[Dict[str, Any]]) -> Dict[str, Any]:
    if not specs: raise RuntimeError("No specs to merge.")
    out = json.loads(json.dumps(specs[0]))
    for s in specs[1:]:
        _deep_merge(out.setdefault("paths", {}), s.get("paths", {}))
        if "components" in s: _deep_merge(out.setdefault("components", {}), s["components"])
        if "definitions" in s: _deep_merge(out.setdefault("definitions", {}), s["definitions"])
        for k in ("parameters","responses","securityDefinitions","securitySchemes","tags"):
            if k in s:
                if isinstance(s[k], dict): _deep_merge(out.setdefault(k, {}), s[k])
                else: out[k] = s[k]
    return out

# ---------------------- Safe resolver ----------------------
class RefResolver:
    def __init__(self, spec: Dict[str, Any]):
        self.spec = spec
        self._cache: Dict[int, Any] = {}
    def _get_by_pointer(self, tokens: List[str]) -> Any:
        cur: Any = self.spec
        for tk in tokens:
            if not isinstance(cur, dict) or tk not in cur: return None
            cur = cur[tk]
        return cur
    def resolve(self, node: Any) -> Any:
        if not isinstance(node, dict): return node
        if id(node) in self._cache: return self._cache[id(node)]
        out: Dict[str, Any] = {}
        self._cache[id(node)] = out
        if "$ref" in node:
            ref = node["$ref"]
            if isinstance(ref, str) and ref.startswith("#/"):
                tgt = self._get_by_pointer(ref[2:].split("/"))
                return tgt if isinstance(tgt, dict) else node
            return node
        for k, v in node.items():
            if isinstance(v, dict): out[k] = self.resolve(v)
            elif isinstance(v, list): out[k] = [self.resolve(x) if isinstance(x, dict) else x for x in v]
            else: out[k] = v
        return out

# ---------------------- Helpers ----------------------
PRIMITIVES = {("string",None):"string", ("integer",None):"number", ("number",None):"number", ("boolean",None):"boolean", ("null",None):"null"}
STRING_FORMATS = {"date-time":"string","date":"string","uuid":"string","email":"string","uri":"string","binary":"Blob | File","byte":"string","password":"string"}
FALLBACK_SCALAR = "unknown"
FALLBACK_INDEX  = "unknown"
MAX_DEPTH = 40

def to_pascal_case(s: str) -> str:
    parts = re.split(r"[^A-Za-z0-9]+", s)
    return "".join(p[:1].upper()+p[1:] for p in parts if p)

def sanitize_ts_identifier(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9_]", "_", s)
    if re.match(r"^\d", s): s = "_"+s
    return s

# ---------- Fingerprint (구조적 해시) & 타입 레지스트리 ----------
CANON_IGNORED_KEYS = {"description","example","examples","deprecated","title","readOnly","writeOnly","x-nullable","default"}

def _schema_fingerprint(schema: Any, *, depth: int = 0, seen: Dict[int, str] = None) -> str:
    """스키마 구조를 문자열로 정규화하여 해시. $ref는 이름만 사용. 순환/깊이 방어."""
    if seen is None: seen = {}
    if depth > MAX_DEPTH: return "DEPTH"
    if not isinstance(schema, dict): return json.dumps(schema, sort_keys=True, ensure_ascii=False)
    if id(schema) in seen: return f"RECUR({seen[id(schema)]})"
    if "$ref" in schema:
        ref = schema["$ref"]
        return f"REF:{ref.split('/')[-1] if isinstance(ref,str) else 'UnknownRef'}"
    tag = f"n{len(seen)}"
    seen[id(schema)] = tag
    items = []
    for k in sorted(schema.keys()):
        if k in CANON_IGNORED_KEYS: continue
        v = schema[k]
        if isinstance(v, dict): items.append((k, _schema_fingerprint(v, depth=depth+1, seen=seen)))
        elif isinstance(v, list):
            items.append((k, "[" + ",".join(_schema_fingerprint(x, depth=depth+1, seen=seen) if isinstance(x, dict) else json.dumps(x, ensure_ascii=False) for x in v) + "]"))
        else:
            items.append((k, json.dumps(v, ensure_ascii=False)))
    canon = json.dumps(items, ensure_ascii=False)
    return hashlib.sha1(canon.encode("utf-8")).hexdigest()

class TypeRegistry:
    """동일 구조를 같은 타입명으로 재사용"""
    def __init__(self):
        self.fp_to_name: Dict[str, str] = {}
        self.auto_idx = 1
    def request_name(self, fp: str, hint: str) -> Tuple[str, bool]:
        if fp in self.fp_to_name:
            return self.fp_to_name[fp], False
        # 새로운 이름 배정
        base = sanitize_ts_identifier(to_pascal_case(hint)) if hint else "AutoType"
        if not base: base = "AutoType"
        name = base
        # 충돌 방지
        while name in self.fp_to_name.values():
            name = f"{base}{self.auto_idx}"; self.auto_idx += 1
        self.fp_to_name[fp] = name
        return name, True

# ---------------------- Schema -> TS ----------------------
def ts_type_for_schema(name_hint, schema, components, resolver, decls, registry: TypeRegistry, depth: int = 0) -> str:
    if depth > MAX_DEPTH: return "unknown /* depth capped */"
    if not isinstance(schema, dict): return FALLBACK_SCALAR

    # $ref는 이름 그대로
    if "$ref" in schema:
        ref = schema["$ref"]; ref_name = ref.split("/")[-1] if isinstance(ref, str) else "UnknownRef"
        return sanitize_ts_identifier(ref_name)

    typ = schema.get("type"); fmt = schema.get("format"); enum = schema.get("enum"); nullable = bool(schema.get("nullable", False))

    # enum
    if enum:
        lit = " | ".join(json.dumps(e, ensure_ascii=False) for e in enum)
        return f"({lit})" + (" | null" if nullable else "")

    # unions
    if "oneOf" in schema:
        union = " | ".join(ts_type_for_schema(name_hint, s, components, resolver, decls, registry, depth+1) for s in schema["oneOf"])
        return f"({union})" + (" | null" if nullable else "")
    if "anyOf" in schema:
        union = " | ".join(ts_type_for_schema(name_hint, s, components, resolver, decls, registry, depth+1) for s in schema["anyOf"])
        return f"({union})" + (" | null" if nullable else "")
    if "allOf" in schema:
        items = [ts_type_for_schema(name_hint, s, components, resolver, decls, registry, depth+1) for s in schema["allOf"]]
        inter = " & ".join(items) if items else "unknown"
        return inter + (" | null" if nullable else "")

    # scalars
    if typ in ("string","integer","number","boolean","null"):
        ts = STRING_FORMATS[fmt] if (typ=="string" and fmt in STRING_FORMATS) else PRIMITIVES.get((typ,None), FALLBACK_SCALAR)
        return ts + (" | null" if nullable else "")

    # arrays
    if typ == "array":
        inner = ts_type_for_schema(name_hint+"Item", schema.get("items", {}), components, resolver, decls, registry, depth+1)
        return f"{inner}[]" + (" | null" if nullable else "")

    # objects — 재사용(중복 제거)
    if typ == "object" or ("properties" in schema) or ("additionalProperties" in schema):
        fp = _schema_fingerprint(schema)
        name, is_new = registry.request_name(fp, name_hint or "AutoType")
        if not is_new:
            return name + (" | null" if nullable else "")

        props = schema.get("properties", {}) or {}
        req = set(schema.get("required", []))
        lines: List[str] = []
        for prop_name, prop_schema in props.items():
            tprop = ts_type_for_schema(to_pascal_case(f"{name}_{prop_name}"), prop_schema, components, resolver, decls, registry, depth+1)
            optional = "?" if prop_name not in req else ""
            safe_key = sanitize_ts_identifier(prop_name)
            lines.append(f"  {safe_key}{optional}: {tprop};")

        addl = schema.get("additionalProperties"); index_sig = ""
        if addl is True:
            index_sig = f"  [key: string]: {FALLBACK_INDEX};\n"
        elif isinstance(addl, dict):
            t_addl = ts_type_for_schema(name+"Additional", addl, components, resolver, decls, registry, depth+1)
            index_sig = f"  [key: string]: {t_addl};\n"

        decl = f"export interface {name} {{\n" + ("\n".join(lines) + ("\n" if lines else "")) + index_sig + "}"
        if decl not in decls: decls.append(decl)
        return name + (" | null" if nullable else "")

    if "title" in schema:
        return sanitize_ts_identifier(to_pascal_case(schema["title"])) + (" | null" if nullable else "")

    return FALLBACK_SCALAR + (" | null" if nullable else "")

# ---------------------- Media helpers ----------------------
JSON_CTYPES = ("application/json","application/ld+json","text/json","application/problem+json")
FORM_CTYPES = ("multipart/form-data","application/x-www-form-urlencoded")

def _first_schema_preferring(ctypes: List[str], content: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    for ct in ctypes:
        if ct in content and isinstance(content[ct], dict):
            media = content[ct]; sch = media.get("schema")
            if sch: return sch
    return None

def _fallback_multipart_schema() -> Dict[str, Any]:
    return {"type":"object","additionalProperties":{"oneOf":[
        {"type":"string"},{"type":"number"},{"type":"boolean"},
        {"type":"string","format":"binary"},
        {"type":"array","items":{"type":"string","format":"binary"}},
        {"type":"null"}
    ]}}

# ---------------------- request/response/params ----------------------
def first_response_schema(op: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    resps = op.get("responses", {}) or {}
    two_xx = [k for k in resps if str(k).startswith("2")]
    ordered = sorted(two_xx) + [k for k in resps if k not in two_xx]
    for code in ordered:
        ro = resps.get(code, {}) or {}
        content = ro.get("content", {}) or {}
        sch = _first_schema_preferring(list(JSON_CTYPES), content)
        if sch: return sch
        for media in content.values():
            if isinstance(media, dict) and "schema" in media and media["schema"]:
                return media["schema"]
    return None

def params_schema(op: Dict[str, Any], resolver: RefResolver) -> Dict[str, Any]:
    params = op.get("parameters", []) or []
    props: Dict[str, Any] = {}; required: List[str] = []
    for p in params:
        p = resolver.resolve(p)
        name = p.get("name")
        sch = p.get("schema") or {"type":"string"}
        if not name: continue
        props[name] = sch
        if p.get("required"): required.append(name)
    return {"type":"object","properties":props,"required":required}

def request_body_schema(op: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    body = op.get("requestBody")
    if not body: return None
    content = (body.get("content") or {})
    sch = _first_schema_preferring(list(JSON_CTYPES), content)
    if sch: return sch
    for ct in FORM_CTYPES:
        if ct in content and isinstance(content[ct], dict):
            media = content[ct]
            if isinstance(media, dict) and "schema" in media and media["schema"]:
                return media["schema"]
            return _fallback_multipart_schema()
    for media in content.values():
        if isinstance(media, dict) and "schema" in media and media["schema"]:
            return media["schema"]
    return None

# ---------------------- Emit ----------------------
HTTP_METHODS = {"get","post","put","patch","delete","head","options","trace"}

def emit_components_schemas(components: Dict[str, Any], resolver: RefResolver, decls: List[str], registry: TypeRegistry) -> None:
    schemas = components.get("schemas", {}) if components else {}
    for raw_name, schema in schemas.items():
        name_hint = sanitize_ts_identifier(to_pascal_case(raw_name))
        ts_expr = ts_type_for_schema(name_hint, schema, components, resolver, decls, registry)
        if ts_expr != name_hint:
            alias = f"export type {name_hint} = {ts_expr};"
            if alias not in decls: decls.append(alias)

def emit_paths(spec: Dict[str, Any], resolver: RefResolver, decls: List[str], registry: TypeRegistry) -> None:
    comp = spec.get("components", {})
    paths = spec.get("paths", {}) or {}
    for raw_path, methods in paths.items():
        if not isinstance(methods, dict): continue
        for m, op in methods.items():
            if m.lower() not in HTTP_METHODS or not isinstance(op, dict): continue
            op_id = op.get("operationId") or f"{m}_{raw_path}"
            base = sanitize_ts_identifier(to_pascal_case(op_id))

            # Params
            pschema = params_schema(op, resolver)
            ptype = ts_type_for_schema(base+"Params", pschema, comp, resolver, decls, registry)
            if ptype != (base+"Params"): decls.append(f"export type {base}Params = {ptype};")

            # Request
            rschema = request_body_schema(op)
            if rschema:
                rtype = ts_type_for_schema(base+"RequestBody", rschema, comp, resolver, decls, registry)
                if rtype != (base+"RequestBody"): decls.append(f"export type {base}RequestBody = {rtype};")
            else:
                decls.append(f"export type {base}RequestBody = undefined;")

            # Response
            retsch = first_response_schema(op)
            if retsch:
                rtype2 = ts_type_for_schema(base+"Response", retsch, comp, resolver, decls, registry)
                if rtype2 != (base+"Response"): decls.append(f"export type {base}Response = {rtype2};")
            else:
                decls.append(f"export type {base}Response = void;")

            decls.append(f"// [{m.upper()}] {raw_path} -> {base}Params, {base}RequestBody, {base}Response")

# ---------------------- Build & Validate ----------------------
STRICT_NO_ANY = True
def assert_no_any(ts_code: str):
    # 타입 문맥의 any만 차단
    pattern = re.compile(r'(?x)(?:[:<|=]\s*any(?=[\s,;>\]\)&]))')
    if pattern.search(ts_code):
        raise RuntimeError("'any' detected **as a type** in output. Adjust mappings or specs.")

def build_types_ts(spec: Dict[str, Any]) -> str:
    resolver = RefResolver(spec)
    decls: List[str] = []
    registry = TypeRegistry()

    header = textwrap.dedent("""
    /* AUTO-GENERATED by OpenAPI -> TS generator (Python)
     * - Merged multiple specs
     * - No 'any' (uses 'unknown' fallback)
     * - Supports JSON, multipart/form-data, x-www-form-urlencoded
     * - Deduplicated reusable types by structural fingerprint
     */
    """).strip()

    comp = spec.get("components", {})
    emit_components_schemas(comp, resolver, decls, registry)
    emit_paths(spec, resolver, decls, registry)

    body = "\n\n".join(OrderedDict.fromkeys(decls))
    code = header + "\n\n" + body + "\n"
    if STRICT_NO_ANY: assert_no_any(code)
    return code

def save_types_ts(ts_code: str, out_path: str = "types2.ts"):
    pathlib.Path(out_path).write_text(ts_code, encoding="utf-8")
    print(f"[ok] wrote {out_path} ({len(ts_code)} bytes)")

# ---------------------- Main ----------------------
if __name__ == "__main__" or True:
    specs = load_all_specs_from_env()
    spec = merge_openapi_specs(specs)
    ts = build_types_ts(spec)
    out = os.getenv("OUT_TYPES", "types2.ts")
    save_types_ts(ts, out)
    print("\n---- preview ----")
    print("\n".join(ts.splitlines()[:80]))


[ok] wrote types2.ts (102050 bytes)

---- preview ----
/* AUTO-GENERATED by OpenAPI -> TS generator (Python)
 * - Merged multiple specs
 * - No 'any' (uses 'unknown' fallback)
 * - Supports JSON, multipart/form-data, x-www-form-urlencoded
 * - Deduplicated reusable types by structural fingerprint
 */

export interface ProfileDto {
  id?: number;
  accountId: number;
  locationId?: number;
  desiredJobCategoryId?: number;
  experienceLevel?: number;
  educationLevel?: number;
  skills?: string;
  desiredLocationId?: number;
  desiredSalaryCode?: number;
  profileImageKey?: string;
  updatedAt?: string;
}

export interface ResultDataProfileDto {
  count?: number;
  msg?: string;
  data?: ProfileDto;
}

export interface ResultDataObjectData {
}

export interface ResultDataObject {
  count?: number;
  msg?: string;
  data?: ResultDataObjectData;
}

export interface ResetPasswordRequestDTO {
  verificationToken?: string;
  code?: string;
  newPassword?: string;
}

export interface ResetPass